# ====================================================================
# CLIMATE SENTIMENT AI - Version avec splits train/val/test et graphiques losses
# Split: 70% train / 10% validation / 20% test
# ====================================================================

In [1]:
pip install streamlit wandb torch transformers peft faiss-cpu sentence-transformers rouge-score plotly pandas tqdm scikit-learn matplotlib numpy==1.26.4


In [2]:
import wandb, os, json, time, torch, numpy as np, pandas as pd, streamlit as st
import random, tqdm
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForSeq2SeqLM,
                          Trainer, TrainingArguments, pipeline, TrainerCallback)
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from sentence_transformers import SentenceTransformer
import faiss
import plotly.graph_objects as go
from rouge_score import rouge_scorer
from sklearn.metrics import accuracy_score, f1_score, classification_report
import matplotlib.pyplot as plt



# ====================================================================
# 1. CONFIGURATION & SETUP
# ====================================================================

In [3]:
# Configuration W&B
WANDB_PROJECT = "climate-sentiment-lora-train-val-test"
if not os.getenv("WANDB_API_KEY"):
    st.error("⚠️ WANDB_API_KEY manquant ! Définissez-le dans vos variables d'environnement.")
    st.stop()

wandb.login(key=os.getenv("WANDB_API_KEY"))

# Configuration modèle - T5 pour classification + génération unifiée
MODEL_NAME = "t5-small"  # Modèle seq2seq pour tâches unifiées
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🔧 Configuration: Modèle={MODEL_NAME}, Device={DEVICE}")

# Optimisations CPU
if DEVICE == "cpu":
    torch.set_num_threads(4)
    print("🚀 Optimisations CPU activées")

# Variables globales pour stocker les métriques d'entraînement
training_logs = {
    "train_loss": [],
    "val_loss": [],
    "test_loss": [],
    "epochs": [],
    "steps": []
}

2025-07-25 14:24:45.860 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:46.408 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-07-25 14:24:46.410 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:46.413 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:46.414 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
wandb: Currently logged in as: k_benyahia (k_benyahia-pstb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


🔧 Configuration: Modèle=t5-small, Device=cuda


# ====================================================================
# 2. CALLBACK POUR TRACKING DES LOSSES
# ====================================================================

In [4]:
class LossTrackingCallback(TrainerCallback):
    """Callback personnalisé pour tracker les losses train/val/test."""

    def __init__(self, test_dataset, tokenizer):
        self.test_dataset = test_dataset
        self.tokenizer = tokenizer

    def on_log(self, args, state, control, logs=None, **kwargs):
        """Appelé à chaque log pendant l'entraînement."""
        if logs:
            current_step = state.global_step

            # Train loss
            if "loss" in logs:
                training_logs["train_loss"].append(logs["loss"])
                training_logs["steps"].append(current_step)

            # Validation loss
            if "eval_loss" in logs:
                training_logs["val_loss"].append(logs["eval_loss"])

                # Calculer test loss si on a des métriques d'évaluation
                if hasattr(state, 'model'):
                    test_loss = self._compute_test_loss(state.model)
                    training_logs["test_loss"].append(test_loss)

    def on_epoch_end(self, args, state, control, **kwargs):
        """Appelé à la fin de chaque époque."""
        training_logs["epochs"].append(state.epoch)

    def _compute_test_loss(self, model):
        """Calcule la loss sur le test set."""
        try:
            model.eval()
            total_loss = 0
            num_batches = 0

            # Échantillonner quelques exemples du test set pour la loss
            test_samples = random.sample(list(self.test_dataset), min(50, len(self.test_dataset)))

            with torch.no_grad():
                for sample in test_samples:
                    inputs = {k: v.unsqueeze(0).to(model.device) for k, v in sample.items()
                             if k in ['input_ids', 'attention_mask', 'labels']}

                    outputs = model(**inputs)
                    total_loss += outputs.loss.item()
                    num_batches += 1

            return total_loss / num_batches if num_batches > 0 else 0

        except Exception as e:
            print(f"Erreur calcul test loss: {e}")
            return 0



# ====================================================================
# 3. FONCTIONS DE VÉRIFICATION (SANITY CHECKS)
# ====================================================================

In [5]:
def sanity_check_dataset(ds, split_name, n=3):
    """Affiche quelques exemples + vérifie présence des colonnes nécessaires."""
    required_cols = ["input_ids", "attention_mask", "labels"]
    for col in required_cols:
        assert col in ds.column_names, f"Colonne {col} manquante dans {split_name}!"

    print(f"📊 Dataset {split_name}: {len(ds)} échantillons")
    for i in range(min(n, len(ds))):
        print(f"Exemple {i}: {ds[i]}")
    print(f"✅ Dataset {split_name} validé")

def sanity_check_model(model, tokenizer, text="Climate change is serious"):
    """Test forward pass rapide."""
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss

    print(f"✅ Forward pass OK - Loss: {loss:.4f}")

def sanity_check_retrieval(encoder, index, corpus, query="climate change"):
    """Test du système de retrieval."""
    if len(corpus) == 0:
        print("❌ Corpus vide")
        return

    emb = encoder.encode([query])
    scores, ids = index.search(np.array(emb).astype("float32"), min(2, len(corpus)))

    print(f"✅ Retrieval OK - Trouvé {len(ids[0])} documents")
    for i, (score, doc_id) in enumerate(zip(scores[0], ids[0])):
        if doc_id < len(corpus):
            print(f"  {i+1}. Score: {score:.3f} - {corpus[doc_id][:100]}...")


# ====================================================================
# 4. CHARGEMENT ET PRÉPARATION DES DONNÉES
# ====================================================================

In [6]:
@st.cache_data
def load_reddit_data(file_path='sentiment_enriched_data_batch.csv', n=10_000):
    """
    Charge et nettoie le dataset Reddit climatique.
    Split: 70% train / 10% validation / 20% test
    """
    try:
        df = pd.read_csv(file_path, quoting=3, on_bad_lines='skip', engine='python') # Ignore quotes and skip bad lines
    except FileNotFoundError:
        st.error(f"❌ Fichier {file_path} non trouvé!")
        return pd.DataFrame()

    # Nettoyage
    df = df.dropna(subset=['comment_sentiment', 'post_title', 'self_text'])

    # Mapping sentiment vers labels numériques
    sentiment2id = {"negative": 0, "neutral": 1, "positive": 2}
    df["label"] = df["comment_sentiment"].map(sentiment2id)
    df = df.dropna(subset=['label'])

    # Création du texte combiné
    df["text"] = (df["post_title"].fillna('') + ' ' + df["self_text"].fillna('')).str.strip()

    # Échantillonnage stratifié
    df_sampled = df.groupby('label').apply(
        lambda x: x.sample(min(len(x), n//3), random_state=42)
    ).reset_index(drop=True)

    print(f"📊 Dataset total: {len(df_sampled)} échantillons")
    print("Distribution des classes:")
    print(df_sampled['label'].value_counts().sort_index())

    return df_sampled[["text", "label", "subreddit"]].reset_index(drop=True)

def create_train_val_test_splits(df, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    """
    Crée les splits train/validation/test avec stratification.

    Args:
        df: DataFrame source
        train_ratio: Proportion pour train (0.7 = 70%)
        val_ratio: Proportion pour validation (0.1 = 10%)
        test_ratio: Proportion pour test (0.2 = 20%)

    Returns:
        dict avec les 3 splits
    """
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Les ratios doivent sommer à 1.0"

    # Première division: 80% (train+val) / 20% (test)
    from sklearn.model_selection import train_test_split

    train_val_df, test_df = train_test_split(
        df,
        test_size=test_ratio,
        random_state=42,
        stratify=df['label']
    )

    # Deuxième division: 70% train / 10% val (sur les 80% restants)
    val_ratio_adjusted = val_ratio / (train_ratio + val_ratio)  # 0.1/0.8 = 0.125

    train_df, val_df = train_test_split(
        train_val_df,
        test_size=val_ratio_adjusted,
        random_state=42,
        stratify=train_val_df['label']
    )

    print(f"📊 Splits créés:")
    print(f"  - Train: {len(train_df)} échantillons ({len(train_df)/len(df)*100:.1f}%)")
    print(f"  - Validation: {len(val_df)} échantillons ({len(val_df)/len(df)*100:.1f}%)")
    print(f"  - Test: {len(test_df)} échantillons ({len(test_df)/len(df)*100:.1f}%)")

    # Vérifier distribution des classes
    print("\n📊 Distribution par split:")
    for split_name, split_df in [("Train", train_df), ("Val", val_df), ("Test", test_df)]:
        dist = split_df['label'].value_counts(normalize=True).sort_index()
        print(f"  {split_name}: {dict(dist)}")

    return {
        "train": train_df.reset_index(drop=True),
        "validation": val_df.reset_index(drop=True),
        "test": test_df.reset_index(drop=True)
    }

# Chargement des données
reddit_df = load_reddit_data()

if len(reddit_df) == 0:
    st.error("❌ Impossible de charger les données")
    st.stop()

# Création des splits
data_splits = create_train_val_test_splits(reddit_df)

# Upload vers W&B
def upload_dataset_to_wandb():
    """Upload du dataset avec splits vers W&B Artifacts."""
    with wandb.init(project=WANDB_PROJECT, job_type="dataset-upload"):
        artifact = wandb.Artifact("reddit-climate-train-val-test", type="dataset")

        # Sauvegarder chaque split
        for split_name, split_df in data_splits.items():
            filename = f"{split_name}_data.csv"
            split_df.to_csv(filename, index=False)
            artifact.add_file(filename)

        # Métadonnées
        metadata = {
            "total_samples": len(reddit_df),
            "train_samples": len(data_splits["train"]),
            "val_samples": len(data_splits["validation"]),
            "test_samples": len(data_splits["test"]),
            "classes": reddit_df['label'].value_counts().to_dict(),
            "split_ratios": {"train": 0.7, "val": 0.1, "test": 0.2}
        }
        artifact.metadata = metadata

        wandb.log_artifact(artifact)
        print("✅ Dataset avec splits uploadé vers W&B")

upload_dataset_to_wandb()

2025-07-25 14:24:47.703 No runtime found, using MemoryCacheStorageManager
2025-07-25 14:24:47.706 No runtime found, using MemoryCacheStorageManager
2025-07-25 14:24:47.707 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:47.708 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:47.708 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:47.709 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:48.218 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:48.221 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:24:48.227 Thread 'Thread-9': missing ScriptRunContext! This warning can be ignored whe

📊 Dataset total: 9999 échantillons
Distribution des classes:
label
0.0    3333
1.0    3333
2.0    3333
Name: count, dtype: int64


2025-07-25 14:26:51.950 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:26:51.951 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:26:51.952 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


📊 Splits créés:
  - Train: 6999 échantillons (70.0%)
  - Validation: 1000 échantillons (10.0%)
  - Test: 2000 échantillons (20.0%)

📊 Distribution par split:
  Train: {0.0: 0.3333333333333333, 1.0: 0.3333333333333333, 2.0: 0.3333333333333333}
  Val: {0.0: 0.334, 1.0: 0.333, 2.0: 0.333}
  Test: {0.0: 0.333, 1.0: 0.3335, 2.0: 0.3335}


✅ Dataset avec splits uploadé vers W&B


# ====================================================================
# 5. TOKENISATION POUR T5 (SEQ2SEQ)
# ====================================================================

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def prepare_seq2seq_data(examples):
    """Prépare les données pour T5 en format seq2seq."""
    inputs = []
    targets = []

    for text, label in zip(examples["text"], examples["label"]):
        input_text = f"classify sentiment: {text}"
        target_text = ["negative", "neutral", "positive"][int(label)]
        inputs.append(input_text)
        targets.append(target_text)

    # Tokenisation
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding=True)
    labels = tokenizer(targets, max_length=16, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Création des datasets pour chaque split
datasets = {}
for split_name, split_df in data_splits.items():
    dataset = Dataset.from_pandas(split_df)
    tokenized = dataset.map(prepare_seq2seq_data, batched=True, remove_columns=dataset.column_names)
    tokenized.set_format(type="torch")
    datasets[split_name] = tokenized

print("✅ Tokenisation terminée pour tous les splits")

# Vérification de chaque split
for split_name, dataset in datasets.items():
    sanity_check_dataset(dataset, split_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/6999 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

✅ Tokenisation terminée pour tous les splits
📊 Dataset train: 6999 échantillons
Exemple 0: {'input_ids': tensor([  853,  4921,  6493,    10,  8586,   172,    22,     7,  1988,    24,
            3,    88,    47,    16,  7764,  7807,   383,  2262,   152,   152,
          904,  7120,  4973,     7,   365,  3044,    57,   245, 10702,    15,
           26,  8468,  2279,   276,  3171,  9135,    10,    37,  2601,    47,
          310,    46, 13876,  2412,   443,    11,    34,    47,    66,    80,
          307,  3562,  1469,     5,     1,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,  

# ====================================================================
# 6. MODÈLE T5 AVEC LORA
# ====================================================================

In [8]:
def setup_lora_model():
    """Configure le modèle T5 avec LoRA pour fine-tuning efficace."""

    base_model = AutoModelForSeq2SeqLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if DEVICE == "cuda" else torch.float32
    ).to(DEVICE)

    lora_config = LoraConfig(
        task_type=TaskType.SEQ_2_SEQ_LM,
        r=16,
        lora_alpha=32,
        target_modules=["q", "v"],
        lora_dropout=0.05,
        bias="none",
    )

    model = get_peft_model(base_model, lora_config)
    model.print_trainable_parameters()

    return model

model = setup_lora_model()
sanity_check_model(model, tokenizer)

trainable params: 589,824 || all params: 61,096,448 || trainable%: 0.9654


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


✅ Forward pass OK - Loss: 0.3286


# ====================================================================
# 7. MÉTRIQUES ET ENTRAÎNEMENT
# ====================================================================


In [9]:
def compute_metrics(eval_pred):
    """Métriques d'évaluation personnalisées."""
    predictions = eval_pred.predictions
    labels = eval_pred.label_ids

    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predicted_tokens = np.argmax(predictions, axis=-1)
    predicted_texts = tokenizer.batch_decode(predicted_tokens, skip_special_tokens=True)
    label_texts = tokenizer.batch_decode(labels, skip_special_tokens=True)

    sentiment_map = {"negative": 0, "neutral": 1, "positive": 2}

    pred_labels = [sentiment_map.get(text.strip().lower(), 1) for text in predicted_texts]
    true_labels = [sentiment_map.get(text.strip().lower(), 1) for text in label_texts]

    accuracy = accuracy_score(true_labels, pred_labels)
    f1 = f1_score(true_labels, pred_labels, average="weighted")

    return {"accuracy": accuracy, "f1": f1}

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./lora_climate_t5_train_val_test",
    per_device_train_batch_size=8 if DEVICE == "cuda" else 4,
    per_device_eval_batch_size=8 if DEVICE == "cuda" else 4,
    num_train_epochs=5,  # Plus d'époques pour voir l'évolution
    learning_rate=5e-4,
    weight_decay=0.01,
    fp16=DEVICE == "cuda",
    eval_strategy="steps",
    eval_steps=50,  # Évaluation plus fréquente
    save_strategy="steps",
    save_steps=100,
    logging_steps=25,  # Logs plus fréquents
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="wandb",
    run_name="lora-t5-train-val-test",
    dataloader_num_workers=0,
)

# Trainer avec callback personnalisé
loss_callback = LossTrackingCallback(datasets["test"], tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"],  # Utilise validation pour l'évaluation pendant training
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[loss_callback]
)

def train_model():
    """Lance l'entraînement avec suivi détaillé des losses."""
    print("🚀 Début de l'entraînement avec tracking train/val/test losses...")

    # Reset des logs
    global training_logs
    training_logs = {"train_loss": [], "val_loss": [], "test_loss": [], "epochs": [], "steps": []}

    with wandb.init(project=WANDB_PROJECT, job_type="training", name="lora-t5-train-val-test"):
        # Entraînement
        trainer.train()

        # Sauvegarde
        trainer.save_model("./lora_climate_t5_train_val_test")
        tokenizer.save_pretrained("./lora_climate_t5_train_val_test")

        # Évaluation finale sur tous les splits
        print("\n📊 Évaluation finale sur tous les splits:")

        train_metrics = trainer.evaluate(datasets["train"])
        val_metrics = trainer.evaluate(datasets["validation"])
        test_metrics = trainer.evaluate(datasets["test"])

        final_metrics = {
            "train_final": train_metrics,
            "val_final": val_metrics,
            "test_final": test_metrics,
            "training_logs": training_logs
        }

        wandb.log(final_metrics)

        print(f"📊 Résultats finaux:")
        print(f"  Train - Accuracy: {train_metrics.get('eval_accuracy', 0):.3f}, F1: {train_metrics.get('eval_f1', 0):.3f}")
        print(f"  Val   - Accuracy: {val_metrics.get('eval_accuracy', 0):.3f}, F1: {val_metrics.get('eval_f1', 0):.3f}")
        print(f"  Test  - Accuracy: {test_metrics.get('eval_accuracy', 0):.3f}, F1: {test_metrics.get('eval_f1', 0):.3f}")

        return final_metrics

/tmp/ipython-input-9-2553260242.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



# ====================================================================
# 8. FONCTIONS DE VISUALISATION DES LOSSES
# ====================================================================

In [10]:
def plot_training_losses():
    """Crée des graphiques interactifs des losses train/val/test."""

    if not training_logs["train_loss"]:
        st.warning("⚠️ Pas de données d'entraînement disponibles. Lancez d'abord l'entraînement.")
        return

    # Graphique principal: Evolution des losses
    fig_losses = go.Figure()

    # Synchroniser les données (toutes les listes doivent avoir la même longueur)
    min_length = min(len(training_logs["train_loss"]),
                     len(training_logs["val_loss"]),
                     len(training_logs["test_loss"]))

    steps = training_logs["steps"][:min_length]

    # Train loss
    fig_losses.add_trace(go.Scatter(
        x=steps,
        y=training_logs["train_loss"][:min_length],
        mode='lines+markers',
        name='Train Loss',
        line=dict(color='blue', width=2),
        marker=dict(size=4)
    ))

    # Validation loss
    fig_losses.add_trace(go.Scatter(
        x=steps,
        y=training_logs["val_loss"][:min_length],
        mode='lines+markers',
        name='Validation Loss',
        line=dict(color='orange', width=2),
        marker=dict(size=4)
    ))

    # Test loss
    fig_losses.add_trace(go.Scatter(
        x=steps,
        y=training_logs["test_loss"][:min_length],
        mode='lines+markers',
        name='Test Loss',
        line=dict(color='red', width=2),
        marker=dict(size=4)
    ))

    fig_losses.update_layout(
        title="📈 Évolution des Losses pendant l'Entraînement",
        xaxis_title="Steps",
        yaxis_title="Loss",
        hovermode='x unified',
        legend=dict(x=0.7, y=0.95),
        height=500
    )

    st.plotly_chart(fig_losses, use_container_width=True)

    # Graphique secondaire: Loss finale par split
    if min_length > 0:
        final_losses = {
            "Train": training_logs["train_loss"][-1] if training_logs["train_loss"] else 0,
            "Validation": training_logs["val_loss"][-1] if training_logs["val_loss"] else 0,
            "Test": training_logs["test_loss"][-1] if training_logs["test_loss"] else 0
        }

        fig_final = go.Figure(data=go.Bar(
            x=list(final_losses.keys()),
            y=list(final_losses.values()),
            marker_color=['blue', 'orange', 'red'],
            text=[f"{v:.4f}" for v in final_losses.values()],
            textposition='auto'
        ))

        fig_final.update_layout(
            title="📊 Losses Finales par Split",
            yaxis_title="Loss",
            height=400
        )

        st.plotly_chart(fig_final, use_container_width=True)

def plot_dataset_distribution():
    """Graphique de distribution des données par split."""

    # Données pour le graphique
    splits_data = []
    for split_name, split_df in data_splits.items():
        for label in [0, 1, 2]:
            count = len(split_df[split_df['label'] == label])
            splits_data.append({
                'Split': split_name.capitalize(),
                'Sentiment': ['Negative', 'Neutral', 'Positive'][label],
                'Count': count
            })

    df_viz = pd.DataFrame(splits_data)

    fig = go.Figure()

    colors = ['#FF4B4B', '#FFA500', '#00CC88']
    sentiments = ['Negative', 'Neutral', 'Positive']

    for i, sentiment in enumerate(sentiments):
        data = df_viz[df_viz['Sentiment'] == sentiment]
        fig.add_trace(go.Bar(
            x=data['Split'],
            y=data['Count'],
            name=sentiment,
            marker_color=colors[i]
        ))

    fig.update_layout(
        title="📊 Distribution des Classes par Split",
        xaxis_title="Split",
        yaxis_title="Nombre d'échantillons",
        barmode='group',
        height=400
    )

    st.plotly_chart(fig, use_container_width=True)


# ====================================================================
# 9. SYSTÈME DE RETRIEVAL (inchangé)
# ====================================================================

In [11]:
@st.cache_resource
def build_retrieval_system():
    """Construction de l'index FAISS pour la recherche sémantique."""
    print("🔍 Construction de l'index de retrieval...")

    corpus = reddit_df["text"].tolist()

    if len(corpus) == 0:
        return None, None, []

    encoder = SentenceTransformer("all-MiniLM-L6-v2")
    embeddings = encoder.encode(corpus, batch_size=32, show_progress_bar=True)

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatIP(dimension)
    index.add(np.array(embeddings).astype("float32"))

    print(f"✅ Index construit: {len(corpus)} documents, dimension {dimension}")

    return encoder, index, corpus

encoder, faiss_index, corpus = build_retrieval_system()

if encoder is not None:
    sanity_check_retrieval(encoder, faiss_index, corpus)

def retrieve_documents(query, k=3, threshold=0.3):
    """Récupère les k documents les plus pertinents pour une query."""
    if encoder is None or faiss_index is None:
        return [], []

    query_embedding = encoder.encode([query])
    scores, indices = faiss_index.search(
        np.array(query_embedding).astype("float32"),
        min(k, len(corpus))
    )

    filtered_results = []
    for score, idx in zip(scores[0], indices[0]):
        if score > threshold and idx < len(corpus):
            filtered_results.append((corpus[idx], score))

    documents = [doc for doc, _ in filtered_results]
    doc_scores = [score for _, score in filtered_results]

    return documents, doc_scores

2025-07-25 14:27:04.246 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:04.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:04.248 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:04.249 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


🔍 Construction de l'index de retrieval...


2025-07-25 14:27:04.754 Thread 'Thread-12': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:04.755 Thread 'Thread-12': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:04.756 Thread 'Thread-12': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2025-07-25 14:27:15.088 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.089 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.090 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


✅ Index construit: 9999 documents, dimension 384
✅ Retrieval OK - Trouvé 2 documents
  1. Score: 0.717 - Cooling the Heat on Climate The propaganda that changed things is when the Left dropped “man-made” f...
  2. Score: 0.703 - How is what we’re doing causing climate change ? I would LOVE to see a source for that....



# ====================================================================
# 10. GÉNÉRATION DE RÉPONSES (inchangé)
# ====================================================================

In [12]:
def classify_sentiment(text):
    """Classification de sentiment avec le modèle fine-tuné."""
    input_text = f"classify sentiment: {text}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=256, truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=20,
            num_beams=3,
            early_stopping=True
        )

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    sentiment_map = {"negative": [0.8, 0.1, 0.1], "neutral": [0.1, 0.8, 0.1], "positive": [0.1, 0.1, 0.8]}
    probs = sentiment_map.get(prediction.lower(), [0.33, 0.34, 0.33])

    return prediction, np.array(probs)

def generate_contextual_response(user_input, context_docs):
    """Génère une réponse contextuelle en utilisant le modèle fine-tuné."""
    context_text = "\n".join([f"- {doc[:200]}..." for doc in context_docs[:3]])
    prompt = f"generate response: User asks about: {user_input}\nContext: {context_text}\nResponse:"

    inputs = tokenizer(prompt, return_tensors="pt", max_length=400, truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=3,
            temperature=0.7,
            do_sample=True,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Response:" in response:
        response = response.split("Response:")[-1].strip()

    return response

# ====================================================================
# 11. INTERFACE STREAMLIT ENRICHIE
# ====================================================================

In [13]:
def main_streamlit_app():
    """Interface utilisateur Streamlit complète avec graphiques de losses."""

    st.set_page_config(
        page_title="🌍 Climate Sentiment AI - Train/Val/Test",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    st.title("🌍 Climate Sentiment AI avec Splits Train/Val/Test")
    st.markdown("*Modèle T5 avec LoRA - Tracking détaillé des losses*")

    # Sidebar avec informations et contrôles
    with st.sidebar:
        st.header("📊 Informations Système")
        st.metric("Modèle", MODEL_NAME)
        st.metric("Device", DEVICE)

        # Métriques des splits
        st.subheader("📂 Splits de Données")
        st.metric("Train", f"{len(data_splits['train'])} (70%)")
        st.metric("Validation", f"{len(data_splits['validation'])} (10%)")
        st.metric("Test", f"{len(data_splits['test'])} (20%)")

        st.divider()

        # Contrôles d'entraînement
        st.subheader("🚀 Entraînement")
        if st.button("🏃 Lancer Entraînement", type="primary"):
            with st.spinner("Entraînement en cours... (plusieurs minutes)"):
                training_results = train_model()
            st.success("✅ Entraînement terminé!")
            st.rerun()  # Rafraîchir pour afficher les graphiques

        if st.button("📊 Évaluer sur Test Set"):
            with st.spinner("Évaluation en cours..."):
                test_metrics = trainer.evaluate(datasets["test"])
            st.success("✅ Évaluation terminée!")
            st.json(test_metrics)

    # Onglets principaux
    tab1, tab2, tab3, tab4 = st.tabs(["🚀 Application", "📈 Losses & Métriques", "📊 Analyse Données", "📚 Documentation"])

    with tab1:
        # Interface de test du modèle
        col1, col2 = st.columns([2, 1])

        with col1:
            st.header("💬 Test du Système")

            user_input = st.text_area(
                "Entrez votre message sur le climat:",
                value="Renewable energy technologies are becoming more affordable and efficient.",
                height=100
            )

            if st.button("🚀 Analyser & Répondre", type="primary"):
                if user_input.strip():

                    # Phase 1: Classification sentiment
                    with st.spinner("🔍 Classification du sentiment..."):
                        sentiment_pred, sentiment_probs = classify_sentiment(user_input)

                    # Affichage sentiment
                    st.subheader("📊 Analyse de Sentiment")
                    fig = go.Figure(data=go.Bar(
                        x=["Négatif", "Neutre", "Positif"],
                        y=sentiment_probs,
                        marker_color=["#FF4B4B", "#FFA500", "#00CC88"]
                    ))
                    fig.update_layout(
                        title=f"Prédiction: {sentiment_pred.capitalize()}",
                        height=300,
                        showlegend=False
                    )
                    st.plotly_chart(fig, use_container_width=True)

                    # Phase 2: Retrieval
                    with st.spinner("🔍 Recherche de contexte pertinent..."):
                        context_docs, context_scores = retrieve_documents(user_input, k=5)

                    if context_docs:
                        st.subheader("📚 Contexte Récupéré")
                        for i, (doc, score) in enumerate(zip(context_docs[:3], context_scores[:3])):
                            with st.expander(f"Document {i+1} (Score: {score:.3f})"):
                                st.write(doc[:300] + "..." if len(doc) > 300 else doc)

                    # Phase 3: Génération
                    with st.spinner("✍️ Génération de la réponse..."):
                        response = generate_contextual_response(user_input, context_docs)

                    st.subheader("🤖 Réponse Générée")
                    st.write(response)

                    # Métriques qualité
                    if context_docs:
                        scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
                        rouge_score = scorer.score(user_input, response)["rougeL"].fmeasure
                        st.metric("Score ROUGE-L", f"{rouge_score:.3f}")

                    # Log interaction vers W&B
                    with wandb.init(project=WANDB_PROJECT, job_type="inference"):
                        wandb.log({
                            "user_input": user_input,
                            "sentiment_prediction": sentiment_pred,
                            "sentiment_probs": sentiment_probs.tolist(),
                            "context_docs": context_docs,
                            "response": response,
                            "num_context_docs": len(context_docs)
                        })
                else:
                    st.warning("⚠️ Veuillez saisir un message.")

        with col2:
            st.header("📈 Métriques & Feedback")

            # Métriques modèle
            st.subheader("🎯 Performance Modèle")
            if training_logs["train_loss"]:
                st.metric("Train Loss (dernière)", f"{training_logs['train_loss'][-1]:.4f}")
                if training_logs["val_loss"]:
                    st.metric("Val Loss (dernière)", f"{training_logs['val_loss'][-1]:.4f}")
                if training_logs["test_loss"]:
                    st.metric("Test Loss (dernière)", f"{training_logs['test_loss'][-1]:.4f}")
            else:
                st.info("Lancez l'entraînement pour voir les métriques")

            # Système de feedback
            st.subheader("💬 Feedback")

            if 'response' in locals():
                feedback_rating = st.radio(
                    "Évaluez la réponse:",
                    options=["👍 Excellente", "👌 Bonne", "👎 Mauvaise"],
                    horizontal=True
                )

                feedback_comment = st.text_area("Commentaire (optionnel):", height=80)

                if st.button("📤 Envoyer Feedback"):
                    feedback_data = {
                        "timestamp": time.time(),
                        "user_input": user_input,
                        "response": response,
                        "rating": feedback_rating,
                        "comment": feedback_comment,
                        "sentiment_pred": sentiment_pred
                    }

                    # Sauvegarde locale
                    with open("feedback_log.jsonl", "a") as f:
                        json.dump(feedback_data, f)
                        f.write("\n")

                    # Log vers W&B
                    with wandb.init(project=WANDB_PROJECT, job_type="feedback"):
                        wandb.log(feedback_data)

                    st.success("✅ Merci pour votre feedback!")

    with tab2:
        st.header("📈 Tracking des Losses et Métriques")

        # Graphiques des losses
        st.subheader("📊 Évolution des Losses pendant l'Entraînement")
        plot_training_losses()

        # Tableau récapitulatif des métriques
        if training_logs["train_loss"]:
            st.subheader("📋 Résumé des Métriques")

            col1, col2, col3 = st.columns(3)

            with col1:
                st.metric(
                    "🔵 Train Loss",
                    f"{training_logs['train_loss'][-1]:.4f}" if training_logs["train_loss"] else "N/A",
                    delta=f"{training_logs['train_loss'][-1] - training_logs['train_loss'][0]:.4f}" if len(training_logs["train_loss"]) > 1 else None
                )

            with col2:
                st.metric(
                    "🟠 Validation Loss",
                    f"{training_logs['val_loss'][-1]:.4f}" if training_logs["val_loss"] else "N/A",
                    delta=f"{training_logs['val_loss'][-1] - training_logs['val_loss'][0]:.4f}" if len(training_logs["val_loss"]) > 1 else None
                )

            with col3:
                st.metric(
                    "🔴 Test Loss",
                    f"{training_logs['test_loss'][-1]:.4f}" if training_logs["test_loss"] else "N/A",
                    delta=f"{training_logs['test_loss'][-1] - training_logs['test_loss'][0]:.4f}" if len(training_logs["test_loss"]) > 1 else None
                )

            # Statistiques détaillées
            if len(training_logs["train_loss"]) > 0:
                st.subheader("📊 Statistiques d'Entraînement")

                stats_data = {
                    "Métrique": ["Train Loss", "Validation Loss", "Test Loss"],
                    "Valeur Initiale": [
                        f"{training_logs['train_loss'][0]:.4f}" if training_logs["train_loss"] else "N/A",
                        f"{training_logs['val_loss'][0]:.4f}" if training_logs["val_loss"] else "N/A",
                        f"{training_logs['test_loss'][0]:.4f}" if training_logs["test_loss"] else "N/A"
                    ],
                    "Valeur Finale": [
                        f"{training_logs['train_loss'][-1]:.4f}" if training_logs["train_loss"] else "N/A",
                        f"{training_logs['val_loss'][-1]:.4f}" if training_logs["val_loss"] else "N/A",
                        f"{training_logs['test_loss'][-1]:.4f}" if training_logs["test_loss"] else "N/A"
                    ],
                    "Amélioration": [
                        f"{((training_logs['train_loss'][0] - training_logs['train_loss'][-1]) / training_logs['train_loss'][0] * 100):.1f}%" if len(training_logs["train_loss"]) > 1 else "N/A",
                        f"{((training_logs['val_loss'][0] - training_logs['val_loss'][-1]) / training_logs['val_loss'][0] * 100):.1f}%" if len(training_logs["val_loss"]) > 1 else "N/A",
                        f"{((training_logs['test_loss'][0] - training_logs['test_loss'][-1]) / training_logs['test_loss'][0] * 100):.1f}%" if len(training_logs["test_loss"]) > 1 else "N/A"
                    ]
                }

                st.dataframe(pd.DataFrame(stats_data), use_container_width=True)
        else:
            st.info("🚀 Lancez l'entraînement pour voir les graphiques de losses")
            st.markdown("""
            **Ce que vous verrez après l'entraînement :**
            - 📈 Graphique interactif des losses train/validation/test
            - 📊 Barres comparatives des losses finales
            - 📋 Tableau récapitulatif avec amélioration en %
            - 📉 Métriques d'overfitting/underfitting
            """)

    with tab3:
        st.header("📊 Analyse des Données")

        # Distribution des splits
        st.subheader("📂 Distribution des Splits")
        plot_dataset_distribution()

        # Statistiques par split
        st.subheader("📈 Statistiques Détaillées")

        splits_stats = []
        for split_name, split_df in data_splits.items():
            stats = {
                "Split": split_name.capitalize(),
                "Total": len(split_df),
                "Negative": len(split_df[split_df['label'] == 0]),
                "Neutral": len(split_df[split_df['label'] == 1]),
                "Positive": len(split_df[split_df['label'] == 2]),
                "Longueur Moyenne": split_df['text'].str.len().mean().round(1),
                "Longueur Médiane": split_df['text'].str.len().median()
            }
            splits_stats.append(stats)

        st.dataframe(pd.DataFrame(splits_stats), use_container_width=True)

        # Analyse de la longueur des textes
        st.subheader("📏 Distribution de la Longueur des Textes")

        fig_lengths = go.Figure()

        for split_name, split_df in data_splits.items():
            lengths = split_df['text'].str.len()
            fig_lengths.add_trace(go.Histogram(
                x=lengths,
                name=split_name.capitalize(),
                opacity=0.7,
                nbinsx=30
            ))

        fig_lengths.update_layout(
            title="Distribution de la Longueur des Textes par Split",
            xaxis_title="Longueur (caractères)",
            yaxis_title="Fréquence",
            barmode='overlay',
            height=400
        )

        st.plotly_chart(fig_lengths, use_container_width=True)

        # Exemples par classe
        st.subheader("📝 Exemples par Classe Sentiment")

        sentiment_names = ["Negative", "Neutral", "Positive"]
        cols = st.columns(3)

        for i, (col, sentiment) in enumerate(zip(cols, sentiment_names)):
            with col:
                st.markdown(f"**{sentiment}**")
                examples = data_splits["train"][data_splits["train"]["label"] == i]["text"].head(3)
                for j, example in enumerate(examples):
                    with st.expander(f"Exemple {j+1}"):
                        st.write(example[:200] + "..." if len(example) > 200 else example)

    with tab4:
        st.header("📚 Documentation Technique")

        st.subheader("🎯 Architecture du Système")
        st.markdown("""
        **Modèle Unifié T5 avec LoRA:**
        - **Base**: T5-small (60M paramètres)
        - **LoRA**: Adaptation efficace avec ~1M paramètres entraînables
        - **Tâches**: Classification de sentiment + Génération contextuelle

        **Splits de Données:**
        - **Train (70%)**: Entraînement principal du modèle
        - **Validation (10%)**: Sélection d'hyperparamètres et early stopping
        - **Test (20%)**: Évaluation finale non biaisée

        **Métriques Trackées:**
        - **Train Loss**: Capacité d'apprentissage sur les données d'entraînement
        - **Validation Loss**: Généralisation pendant l'entraînement
        - **Test Loss**: Performance finale sur données non vues
        """)

        st.subheader("📈 Interprétation des Graphiques")
        st.markdown("""
        **Signaux Positifs:**
        - 📉 Train et Val loss diminuent ensemble
        - 🎯 Écart raisonnable entre Train et Val loss
        - ✅ Test loss proche de la Val loss

        **Signaux d'Alerte:**
        - 📈 Val loss augmente alors que Train loss diminue (overfitting)
        - 📊 Écart très important Train vs Val loss (overfitting)
        - ⚠️ Test loss beaucoup plus élevée que Val loss (data leakage)

        **Actions Correctives:**
        - **Overfitting**: Réduire learning rate, augmenter regularization
        - **Underfitting**: Augmenter complexité modèle, plus d'époques
        - **Data leakage**: Vérifier stratification des splits
        """)

        st.subheader("🔧 Configuration LoRA")
        st.code("""
        LoraConfig(
            task_type=TaskType.SEQ_2_SEQ_LM,
            r=16,                    # Rang des matrices (compromis capacité/efficacité)
            lora_alpha=32,           # Facteur de scaling (généralement 2*r)
            target_modules=["q", "v"], # Modules d'attention adaptés
            lora_dropout=0.05,       # Régularisation
            bias="none"              # Pas d'adaptation des biais
        )
        """, language="python")

        st.subheader("⚡ Optimisations Performance")
        st.markdown("""
        **Mémoire:**
        - FP16 automatique sur GPU
        - Batch sizes adaptatives selon hardware
        - Gradient accumulation si nécessaire

        **Calcul:**
        - Multi-threading contrôlé sur CPU
        - Evaluation stratégique (tous les 50 steps)
        - Caching Streamlit pour éviter recomputation

        **I/O:**
        - Tokenisation en batch
        - Sauvegarde périodique des checkpoints
        - Logs structurés vers W&B
        """)


# ====================================================================
# 12. ÉVALUATION END-TO-END AMÉLIORÉE
# ====================================================================


In [14]:
def evaluate_system_performance_detailed(n_samples=50):
    """
    Évaluation complète du système avec métriques détaillées par split.
    """
    print(f"📊 Évaluation détaillée sur {n_samples} échantillons par split...")

    results_by_split = {}

    # Évaluer chaque split séparément
    for split_name, split_df in data_splits.items():
        if len(split_df) == 0:
            continue

        # Échantillonner
        test_samples = split_df.sample(n=min(n_samples, len(split_df)), random_state=42)

        split_results = {
            "sentiment_predictions": [],
            "true_sentiments": [],
            "retrieval_scores": [],
            "rouge_scores": [],
            "latencies": []
        }

        print(f"🔍 Évaluation sur {split_name} ({len(test_samples)} échantillons)...")

        for idx, row in tqdm.tqdm(test_samples.iterrows(), total=len(test_samples)):
            start_time = time.time()

            # Classification sentiment
            pred_sentiment, _ = classify_sentiment(row["text"])
            true_sentiment = ["negative", "neutral", "positive"][row["label"]]

            # Retrieval
            docs, scores = retrieve_documents(row["text"], k=3)
            avg_retrieval_score = np.mean(scores) if scores else 0

            # Génération
            response = generate_contextual_response(row["text"], docs) if docs else "No context available."

            # ROUGE
            scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
            rouge_score = scorer.score(row["text"], response)["rougeL"].fmeasure

            latency = time.time() - start_time

            # Stockage
            split_results["sentiment_predictions"].append(pred_sentiment)
            split_results["true_sentiments"].append(true_sentiment)
            split_results["retrieval_scores"].append(avg_retrieval_score)
            split_results["rouge_scores"].append(rouge_score)
            split_results["latencies"].append(latency)

        # Calcul métriques du split
        sentiment_accuracy = accuracy_score(split_results["true_sentiments"], split_results["sentiment_predictions"])

        split_metrics = {
            "sentiment_accuracy": sentiment_accuracy,
            "avg_retrieval_relevance": np.mean(split_results["retrieval_scores"]),
            "avg_rouge_l": np.mean(split_results["rouge_scores"]),
            "avg_latency_seconds": np.mean(split_results["latencies"]),
            "total_samples": len(test_samples),
            "detailed_results": split_results
        }

        results_by_split[split_name] = split_metrics

        print(f"✅ {split_name} - Accuracy: {sentiment_accuracy:.3f}")

    # Log complet vers W&B
    with wandb.init(project=WANDB_PROJECT, job_type="detailed-evaluation"):
        for split_name, metrics in results_by_split.items():
            # Métriques générales
            wandb.log({f"{split_name}_{k}": v for k, v in metrics.items() if k != "detailed_results"})

            # Table détaillée
            detailed = metrics["detailed_results"]
            eval_table = wandb.Table(
                columns=["split", "true_sentiment", "pred_sentiment", "retrieval_score", "rouge_score", "latency"],
                data=list(zip(
                    [split_name] * len(detailed["true_sentiments"]),
                    detailed["true_sentiments"],
                    detailed["sentiment_predictions"],
                    detailed["retrieval_scores"],
                    detailed["rouge_scores"],
                    detailed["latencies"]
                ))
            )
            wandb.log({f"{split_name}_detailed_results": eval_table})

    return results_by_split

# ====================================================================
# 13. POINT D'ENTRÉE PRINCIPAL
# ====================================================================

In [15]:
def main():
    """Point d'entrée principal avec gestion des splits train/val/test."""

    # Vérifications initiales
    if len(reddit_df) == 0:
        st.error("❌ Données non disponibles. Vérifiez le fichier CSV.")
        return

    # Interface principale
    main_streamlit_app()


# ====================================================================
# 14. FONCTIONS UTILITAIRES POUR ANALYSE AVANCÉE
# ====================================================================


In [16]:
def analyze_overfitting():
    """Analyse des signes d'overfitting/underfitting."""
    st.header("🔍 Analyse Overfitting/Underfitting")

    if not training_logs["train_loss"] or not training_logs["val_loss"]:
        st.warning("⚠️ Données d'entraînement insuffisantes pour l'analyse")
        return

    # Calculer métriques d'overfitting
    final_train_loss = training_logs["train_loss"][-1]
    final_val_loss = training_logs["val_loss"][-1]

    overfitting_ratio = final_val_loss / final_train_loss if final_train_loss > 0 else 1

    col1, col2, col3 = st.columns(3)

    with col1:
        st.metric("🔵 Train Loss Finale", f"{final_train_loss:.4f}")

    with col2:
        st.metric("🟠 Val Loss Finale", f"{final_val_loss:.4f}")

    with col3:
        color = "normal"
        if overfitting_ratio > 1.5:
            color = "inverse"  # Rouge pour overfitting
        elif overfitting_ratio < 0.8:
            color = "inverse"  # Rouge pour comportement anormal

        st.metric(
            "📊 Ratio Val/Train",
            f"{overfitting_ratio:.2f}",
            delta=f"{'⚠️ Overfitting' if overfitting_ratio > 1.5 else '✅ OK' if 0.8 <= overfitting_ratio <= 1.5 else '⚠️ Underfitting'}"
        )

    # Diagnostic automatique
    st.subheader("🔍 Diagnostic Automatique")

    if overfitting_ratio > 1.5:
        st.error("""
        🚨 **Signes d'Overfitting Détectés**

        **Recommandations:**
        - Réduire le learning rate
        - Augmenter la régularisation (dropout)
        - Réduire le nombre d'époques
        - Augmenter la taille du dataset de validation
        """)
    elif overfitting_ratio < 0.8:
        st.warning("""
        ⚠️ **Comportement Anormal Détecté**

        **Possibles causes:**
        - Data leakage entre train et validation
        - Validation set trop facile
        - Problème dans le calcul des métriques
        """)
    else:
        st.success("""
        ✅ **Entraînement Sain**

        Le modèle montre une bonne généralisation avec un écart raisonnable
        entre les performances train et validation.
        """)

def export_training_report():
    """Génère un rapport complet d'entraînement."""
    st.header("📄 Rapport d'Entraînement")

    if st.button("📋 Générer Rapport Complet"):

        # Collecter toutes les informations
        report_data = {
            "configuration": {
                "model_name": MODEL_NAME,
                "device": DEVICE,
                "dataset_size": len(reddit_df),
                "train_size": len(data_splits["train"]),
                "val_size": len(data_splits["validation"]),
                "test_size": len(data_splits["test"])
            },
            "training_results": training_logs,
            "splits_distribution": {
                split_name: split_df['label'].value_counts().to_dict()
                for split_name, split_df in data_splits.items()
            }
        }

        # Sauvegarder en JSON
        report_filename = f"training_report_{int(time.time())}.json"
        with open(report_filename, "w") as f:
            json.dump(report_data, f, indent=2)

        st.success(f"✅ Rapport sauvegardé: {report_filename}")

        # Afficher résumé
        st.subheader("📊 Résumé du Rapport")
        st.json({
            "Total Samples": len(reddit_df),
            "Training Steps": len(training_logs["steps"]) if training_logs["steps"] else 0,
            "Best Train Loss": min(training_logs["train_loss"]) if training_logs["train_loss"] else "N/A",
            "Best Val Loss": min(training_logs["val_loss"]) if training_logs["val_loss"] else "N/A",
            "Model Saved": os.path.exists("./lora_climate_t5_train_val_test")
        })

# ====================================================================
# 15. LANCEMENT DE L'APPLICATION
# ====================================================================

In [17]:
if __name__ == "__main__":
    # Vérification des dépendances
    try:
        import streamlit as st
        import wandb
        import torch
        import transformers
        import peft
        import faiss
        import sentence_transformers
        from sklearn.model_selection import train_test_split

        print("✅ Toutes les dépendances sont installées")

    except ImportError as e:
        print(f"❌ Dépendance manquante: {e}")
        print("Installez avec: pip install streamlit wandb torch transformers peft faiss-cpu sentence-transformers rouge-score plotly pandas numpy tqdm scikit-learn")
        exit(1)

    # Configuration logging
    import logging
    logging.basicConfig(level=logging.INFO)

    # Message de bienvenue
    print("\n" + "="*70)
    print("🌍 CLIMATE SENTIMENT AI - VERSION TRAIN/VAL/TEST")
    print("="*70)
    print(f"📱 Interface: Streamlit avec graphiques de losses")
    print(f"🤖 Modèle: {MODEL_NAME} avec LoRA")
    print(f"💾 Device: {DEVICE}")
    print(f"📊 Splits: Train({len(data_splits['train'])}), Val({len(data_splits['validation'])}), Test({len(data_splits['test'])})")
    print(f"🔍 Corpus: {len(corpus) if corpus else 0} documents indexés")
    print("="*70)

    # Lancement de l'interface
    main()

    # Options avancées dans la sidebar
    with st.sidebar:
        st.divider()
        st.header("🔧 Analyses Avancées")

        if st.button("🔍 Analyser Overfitting"):
            analyze_overfitting()

        if st.button("📄 Exporter Rapport"):
            export_training_report()

        if st.button("📊 Évaluation Détaillée"):
            with st.spinner("Évaluation détaillée en cours..."):
                detailed_results = evaluate_system_performance_detailed(30)
            st.success("✅ Évaluation terminée!")
            st.json({split: {k: v for k, v in metrics.items() if k != "detailed_results"}
                    for split, metrics in detailed_results.items()})

        # Informations debug
        with st.expander("🐛 Debug Info"):
            st.write("**Statut Modèle:**")
            if 'model' in globals():
                trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
                total_params = sum(p.numel() for p in model.parameters())
                st.write(f"- Paramètres entraînables: {trainable_params:,}")
                st.write(f"- Paramètres totaux: {total_params:,}")
                st.write(f"- Ratio LoRA: {trainable_params/total_params*100:.2f}%")

            st.write("**Statut Données:**")
            for split_name, split_data in datasets.items():
                st.write(f"- {split_name}: {len(split_data)} échantillons")

            st.write("**Logs Entraînement:**")
            st.write(f"- Steps trackés: {len(training_logs['steps'])}")
            st.write(f"- Train losses: {len(training_logs['train_loss'])}")
            st.write(f"- Val losses: {len(training_logs['val_loss'])}")
            st.write(f"- Test losses: {len(training_logs['test_loss'])}")

2025-07-25 14:27:15.220 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.224 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.225 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.226 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.228 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.230 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

✅ Toutes les dépendances sont installées

🌍 CLIMATE SENTIMENT AI - VERSION TRAIN/VAL/TEST
📱 Interface: Streamlit avec graphiques de losses
🤖 Modèle: t5-small avec LoRA
💾 Device: cuda
📊 Splits: Train(6999), Val(1000), Test(2000)
🔍 Corpus: 9999 documents indexés


2025-07-25 14:27:15.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.350 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.351 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.352 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.353 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-25 14:27:15.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [18]:
# ====================================================================
# 16. INSTRUCTIONS DE DÉPLOIEMENT MISES À JOUR
# ====================================================================

"""
INSTRUCTIONS DE DÉPLOIEMENT - VERSION TRAIN/VAL/TEST:

1. **Installation des dépendances:**
   ```bash
   pip install streamlit wandb torch transformers peft faiss-cpu sentence-transformers rouge-score plotly pandas numpy tqdm scikit-learn matplotlib
   ```

2. **Configuration W&B:**
   ```bash
   export WANDB_API_KEY="votre_clé_api_wandb"
   ```

3. **Préparation des données:**
   - Placez 'sentiment_enriched_data_batch.csv' dans le répertoire
   - Le script créera automatiquement les splits 70/10/20

4. **Lancement:**
   ```bash
   streamlit run votre_script.py
   ```

5. **Nouvelles fonctionnalités:**
   - 📈 Graphiques interactifs train/val/test losses
   - 📊 Analyse automatique d'overfitting
   - 📋 Rapports d'entraînement détaillés
   - 🔍 Évaluation par split
   - 📉 Métriques de convergence

6. **Workflow recommandé:**
   - Vérifiez la distribution des données (onglet Analyse)
   - Lancez l'entraînement (sidebar)
   - Surveillez les losses en temps réel (onglet Losses)
   - Analysez l'overfitting (sidebar avancé)
   - Évaluez sur le test set final

La version inclut un tracking complet des métriques avec détection
automatique des problèmes d'entraînement et recommandations.
"""

'\nINSTRUCTIONS DE DÉPLOIEMENT - VERSION TRAIN/VAL/TEST:\n\n1. **Installation des dépendances:**\n   ```bash\n   pip install streamlit wandb torch transformers peft faiss-cpu sentence-transformers rouge-score plotly pandas numpy tqdm scikit-learn matplotlib\n   ```\n\n2. **Configuration W&B:**\n   ```bash\n   export WANDB_API_KEY="votre_clé_api_wandb"\n   ```\n\n3. **Préparation des données:**\n   - Placez \'sentiment_enriched_data_batch.csv\' dans le répertoire\n   - Le script créera automatiquement les splits 70/10/20\n\n4. **Lancement:**\n   ```bash\n   streamlit run votre_script.py\n   ```\n\n5. **Nouvelles fonctionnalités:**\n   - 📈 Graphiques interactifs train/val/test losses\n   - 📊 Analyse automatique d\'overfitting\n   - 📋 Rapports d\'entraînement détaillés\n   - 🔍 Évaluation par split\n   - 📉 Métriques de convergence\n\n6. **Workflow recommandé:**\n   - Vérifiez la distribution des données (onglet Analyse)\n   - Lancez l\'entraînement (sidebar)\n   - Surveillez les losses en 